<a href="https://colab.research.google.com/github/heber-augusto/sus-kpis-analysis/blob/main/sia/template_spark_gcs_access_delta_lake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuração do ambiente

**Lembre-se de baixar o arquivo json que garante acesso ao google storage**

In [1]:
!rm -r sus-kpis-analysis
!git clone https://github.com/heber-augusto/sus-kpis-analysis.git

rm: cannot remove 'sus-kpis-analysis': No such file or directory
Cloning into 'sus-kpis-analysis'...
remote: Enumerating objects: 616, done.
remote: Counting objects: 100% (399/399), done.
remote: Compressing objects: 100% (258/258), done.
remote: Total 616 (delta 202), reused 196 (delta 91), pack-reused 217
Receiving objects: 100% (616/616), 2.85 MiB | 2.76 MiB/s, done.
Resolving deltas: 100% (296/296), done.


## Instalação de libs Python, inicialização de variáveis de ambiente e configuração/instalação do Spark

In [2]:
!pip install -r /content/sus-kpis-analysis/sia/etls/requirements.txt

%env PYTHONHASHSEED=1234
%env JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
%env SPARK_HOME=/content/spark-3.4.1-bin-hadoop3
%env SPARK_VERSION=3.4.1

!source /content/sus-kpis-analysis/sia/etls/bin/setup_spark_env.sh '/content/'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=f1b8a68cba2c2d21d5ad69eea6f406fabe6fd5673417ff45a3f6f5e6995376d3
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
env: PYTHONHASHSEED=1234
env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64
env: SPARK_HOME=/content/spark-3.4.1-bin-hadoop3
env: SPARK_VERSION=3.4.1
starting spark env setup 
installing and downloading packages
setting enviroment variables


## Inicializa variáveis de acesso ao delta lake criado no google storage

- Local do arquivo de credencial do storage, diretorio do warehouse e path do spark:

> O arquivo json_file_name deve ser enviado para o ambiente e deve ser utilizado um com as devidas permissões (em caso de escrita)

> O caminho do warehouse pode ser alterado em caso de testes de escritas locais.

> O caminho do spark é setado pelo script de configuração

In [3]:
import os

# id do projeto
project_id = 'teak-ellipse-317015'
# id do bucket dentro do projeto
bucket_id = 'observatorio-oncologia'

# nome da pasta do projeto
project_folder_name = 'monitor'

dev_lake_name = "lake-rosa-dev"

lake_zone = "silver"

database_name = "cancer_data"


json_filename = r'/content/gcp-leitura.json'
# json_filename = r'/content/gcp-escrita.json'


# warehouse_dir = f"gs://{bucket_id}/{dev_lake_name}/"
warehouse_dir = f"/content/{dev_lake_name}/"

spark_path = os.getenv('SPARK_HOME')
spark_path

'/content/spark-3.4.1-bin-hadoop3'

## Inclusão da pasta do repositório no python path

Procedimento permite que funções e classes presentes no repositório sejam utilizadas

In [4]:
import sys
sys.path.append('/content/sus-kpis-analysis')
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/sus-kpis-analysis']

## Importação de funções utilizadas pelo código

In [5]:
from sia.etls.lib.catalog_loader import DeltaLakeDatabaseGsCreator
from sia.etls.lib.table_utilities import vacuum_tables_from_database
from sia.etls.lib.gs_spark_session import create_gs_spark_session

## Cria Sessão Spark conectada ao Delta Lake presente no Google Storage

In [6]:
spark = create_gs_spark_session(
    gcs_project_id = project_folder_name,
    gcs_json_keyfile=json_filename,
    warehouse_dir=warehouse_dir,
    spark_path=spark_path
)


## Refresh do catálogo para utilizar consultas

In [7]:
# Cliente do Google Cloud Storage
from google.cloud import storage
storage_client = storage.Client.from_service_account_json(json_filename)

database_location = f'{dev_lake_name}/{lake_zone}'  # Substitua com o local do seu banco de dados Delta Lake

db_creator = DeltaLakeDatabaseGsCreator(
    spark_session = spark,
    storage_client = storage_client,
    gs_bucket_id = bucket_id,
    database_location = database_location,
    database_name = database_name)
db_creator.create_database(use_db_folder_path = False)
db_creator.recreate_tables()

Banco de dados cancer_data criado.
listando conteúdos do bucket observatorio-oncologia do caminho lake-rosa-dev/silver e database cancer_data
Tabela aq_filtered criada
Tabela ar_filtered criada
Tabela cadastro_municipios criada
Tabela dados_estados_mensal criada
Tabela dados_municipios_mensal criada
Tabela demografia_municipios criada
Tabela pacientes criada
Tabela procedimentos criada
Tabela procedimentos_e_pacientes criada
Recriação das tabelas concluída.


# Exemplos de consultas

## Listagem de bancos e tabelas

In [8]:
spark.sql(f"SHOW DATABASES;").show()

+-----------+
|  namespace|
+-----------+
|cancer_data|
|    default|
+-----------+



In [10]:
spark.sql(f"SHOW TABLES FROM {database_name};").show()

+-----------+--------------------+-----------+
|  namespace|           tableName|isTemporary|
+-----------+--------------------+-----------+
|cancer_data|         aq_filtered|      false|
|cancer_data|         ar_filtered|      false|
|cancer_data| cadastro_municipios|      false|
|cancer_data|dados_estados_mensal|      false|
|cancer_data|dados_municipios_...|      false|
|cancer_data|demografia_munici...|      false|
|cancer_data|           pacientes|      false|
|cancer_data|       procedimentos|      false|
|cancer_data|procedimentos_e_p...|      false|
+-----------+--------------------+-----------+



## Consultas em dados de cidades

In [9]:
spark.sql(f"SELECT * FROM {database_name}.cadastro_municipios").show()

+-------+--------------------+-----+--------+
|     id|                nome|id_uf| nome_uf|
+-------+--------------------+-----+--------+
|1100015|Alta Floresta D'O...|   11|Rondônia|
|1100023|           Ariquemes|   11|Rondônia|
|1100031|              Cabixi|   11|Rondônia|
|1100049|              Cacoal|   11|Rondônia|
|1100056|          Cerejeiras|   11|Rondônia|
|1100064|   Colorado do Oeste|   11|Rondônia|
|1100072|          Corumbiara|   11|Rondônia|
|1100080|       Costa Marques|   11|Rondônia|
|1100098|     Espigão D'Oeste|   11|Rondônia|
|1100106|       Guajará-Mirim|   11|Rondônia|
|1100114|                Jaru|   11|Rondônia|
|1100122|           Ji-Paraná|   11|Rondônia|
|1100130|  Machadinho D'Oeste|   11|Rondônia|
|1100148|Nova Brasilândia ...|   11|Rondônia|
|1100155| Ouro Preto do Oeste|   11|Rondônia|
|1100189|       Pimenta Bueno|   11|Rondônia|
|1100205|         Porto Velho|   11|Rondônia|
|1100254|   Presidente Médici|   11|Rondônia|
|1100262|          Rio Crespo|   1

In [11]:
spark.sql(f"SELECT * FROM {database_name}.demografia_municipios").show()

+-----------+----------+---------------+--------------+------------+--------------------+----+---------+-----------+
|codigo_sexo|valor_sexo|codigo_situacao|valor_situacao|codigo_local|          nome_local| ano|populacao|faixa_idade|
+-----------+----------+---------------+--------------+------------+--------------------+----+---------+-----------+
|          4|    Homens|              0|         Total|     1100015|Alta Floresta D'O...|2010|    12656|      Total|
|          4|    Homens|              0|         Total|     1100023|      Ariquemes - RO|2010|    45543|      Total|
|          4|    Homens|              0|         Total|     1100031|         Cabixi - RO|2010|     3266|      Total|
|          4|    Homens|              0|         Total|     1100049|         Cacoal - RO|2010|    39124|      Total|
|          4|    Homens|              0|         Total|     1100056|     Cerejeiras - RO|2010|     8551|      Total|
|          4|    Homens|              0|         Total|     1100